In [26]:
import yfinance as yf
import pandas as pd
import statsmodels.api as sm
import numpy as np


In [2]:
# reading in social scores
data_path = ''
social_scores = pd.read_csv(data_path+'social_scores.csv', index_col=0)

In [3]:
# reading in company-ticker mapping
df_financial = pd.read_csv('Financial_data/Tickers.csv', sep=';')
df_financial.head()

,Company,Ticker
0,3M,MMM
1,Abbott,ABT
2,Accenture,ACN
3,Adobe,ADBE
4,AEScorporation,AES


In [4]:
# merging financial and social_scores df
df_financial = df_financial.merge(social_scores, on='Company')
df_financial.head()

,Company,Ticker,scores_2016,scores_2017,scores_2018,scores_2019,scores_2020,total_2016,total_2017,total_2018,total_2019,total_2020
0,3M,MMM,1745,1535,1806,1809,2096,7537,8099,8215,7996,9147
1,Abbott,ABT,1335,1913,918,1144,1602,5719,7288,5201,6317,11414
2,Accenture,ACN,822,919,802,865,210,2293,2526,1825,2570,435
3,Adobe,ADBE,80,99,124,185,222,409,499,616,818,940
4,AEScorporation,AES,2,165,524,518,199,117,1041,2125,3049,18734


In [5]:
#Adding Total Revenue and Market cap for each year
manual_data = pd.read_csv('Financial_data/Manual_finance_data.csv', sep=';')

df_financial = df_financial.merge(manual_data, on='Ticker')
df_financial.head()

,Company,Ticker,scores_2016,scores_2017,scores_2018,scores_2019,scores_2020,total_2016,total_2017,total_2018,...,Total_revenue_2020,Total_revenue_2021,Market_cap_2017,Market_cap_2018,Market_cap_2019,Market_cap_2020,Market_cap_2021,Market_cap_2022,Unnamed: 13,Unnamed: 14
0,3M,MMM,1745,1535,1806,1809,2096,7537,8099,8215,...,32184,35355,107404,140188,110949,101450,100823,102360,NaN,NaN
1,Abbott,ABT,1335,1913,918,1144,1602,5719,7288,5201,...,34608,43075,60674,99316,127036,153608,187600,241365,NaN,NaN
2,Accenture,ACN,822,919,802,865,210,2293,2526,1825,...,44327,50533,74083,90984,105120,127749,157815,225748,NaN,NaN
3,Adobe,ADBE,80,99,124,185,222,409,499,616,...,12868,15785,49589,88493,122468,149837,228840,293345,NaN,NaN
4,AEScorporation,AES,2,165,524,518,199,117,1041,2125,...,9660,11141,7660,7152,9577,13211,15631,16201,NaN,NaN


In [6]:
# dropping columns that were added through merge
df_financial.drop(columns=['Unnamed: 13', 'Unnamed: 14'], inplace=True)
df_financial.columns

Index(['Company', 'Ticker', 'scores_2016', 'scores_2017', 'scores_2018',
       'scores_2019', 'scores_2020', 'total_2016', 'total_2017', 'total_2018',
       'total_2019', 'total_2020', 'Total_revenue_2016', 'Total_revenue_2017',
       'Total_revenue_2018', 'Total_revenue_2019', 'Total_revenue_2020',
       'Total_revenue_2021', 'Market_cap_2017', 'Market_cap_2018',
       'Market_cap_2019', 'Market_cap_2020', 'Market_cap_2021',
       'Market_cap_2022'],
      dtype='object')

In [7]:
#Calculating Revnue growth from total revenue
df_financial['Revenue_growth_2017'] = (df_financial['Total_revenue_2017']/df_financial['Total_revenue_2016'])-1
df_financial['Revenue_growth_2018'] = (df_financial['Total_revenue_2018']/df_financial['Total_revenue_2017'])-1
df_financial['Revenue_growth_2019'] = (df_financial['Total_revenue_2019']/df_financial['Total_revenue_2018'])-1
df_financial['Revenue_growth_2020'] = (df_financial['Total_revenue_2020']/df_financial['Total_revenue_2019'])-1
df_financial['Revenue_growth_2021'] = (df_financial['Total_revenue_2021']/df_financial['Total_revenue_2020'])-1

# Calculating average Market cap for each year (Market_Cap_2017 is from 01.01.2017 etc.)
df_financial['Avg_market_cap_2017'] = (df_financial['Market_cap_2017']+df_financial['Market_cap_2018'])/2
df_financial['Avg_market_cap_2018'] = (df_financial['Market_cap_2018']+df_financial['Market_cap_2019'])/2
df_financial['Avg_market_cap_2019'] = (df_financial['Market_cap_2019']+df_financial['Market_cap_2020'])/2
df_financial['Avg_market_cap_2020'] = (df_financial['Market_cap_2020']+df_financial['Market_cap_2021'])/2
df_financial['Avg_market_cap_2021'] = (df_financial['Market_cap_2021']+df_financial['Market_cap_2022'])/2

df_financial.head()

,Company,Ticker,scores_2016,scores_2017,scores_2018,scores_2019,scores_2020,total_2016,total_2017,total_2018,...,Revenue_growth_2017,Revenue_growth_2018,Revenue_growth_2019,Revenue_growth_2020,Revenue_growth_2021,Avg_market_cap_2017,Avg_market_cap_2018,Avg_market_cap_2019,Avg_market_cap_2020,Avg_market_cap_2021
0,3M,MMM,1745,1535,1806,1809,2096,7537,8099,8215,...,0.051413,0.035000,-0.019197,0.001494,0.098527,123796.0,125568.5,106199.5,101136.5,101591.5
1,Abbott,ABT,1335,1913,918,1144,1602,5719,7288,5201,...,0.313480,0.116393,0.043365,0.084754,0.244654,79995.0,113176.0,140322.0,170604.0,214482.5
2,Accenture,ACN,822,919,802,865,210,2293,2526,1825,...,0.039630,0.133127,0.054230,0.025732,0.140005,82533.5,98052.0,116434.5,142782.0,191781.5
3,Adobe,ADBE,80,99,124,185,222,409,499,616,...,0.307486,0.236817,0.237099,0.151911,0.226686,69041.0,105480.5,136152.5,189338.5,261092.5
4,AEScorporation,AES,2,165,524,518,199,117,1041,2125,...,0.024219,0.019563,-0.050950,-0.051919,0.153313,7406.0,8364.5,11394.0,14421.0,15916.0


In [9]:
# extracting adujsted price for each year
df_financial['adjusted_01.01.2017'] = df_financial['Ticker'].apply(lambda x: yf.download(x, '2017-01-01',progress=False)['Adj Close'].values[0])
df_financial['adjusted_01.01.2018'] = df_financial['Ticker'].apply(lambda x: yf.download(x, '2018-01-01',progress=False)['Adj Close'].values[0])
df_financial['adjusted_01.01.2019'] = df_financial['Ticker'].apply(lambda x: yf.download(x, '2019-01-01',progress=False)['Adj Close'].values[0])
df_financial['adjusted_01.01.2020'] = df_financial['Ticker'].apply(lambda x: yf.download(x, '2020-01-01',progress=False)['Adj Close'].values[0])
df_financial['adjusted_01.01.2021'] = df_financial['Ticker'].apply(lambda x: yf.download(x, '2021-01-01',progress=False)['Adj Close'].values[0])
df_financial['adjusted_01.01.2022'] = df_financial['Ticker'].apply(lambda x: yf.download(x, '2022-01-01',progress=False)['Adj Close'].values[0])

In [12]:
# calculating returns
df_financial['return_2017'] = df_financial['adjusted_01.01.2018']/df_financial['adjusted_01.01.2017']-1
df_financial['return_2018'] = df_financial['adjusted_01.01.2019']/df_financial['adjusted_01.01.2018']-1
df_financial['return_2019'] = df_financial['adjusted_01.01.2020']/df_financial['adjusted_01.01.2019']-1
df_financial['return_2020'] = df_financial['adjusted_01.01.2021']/df_financial['adjusted_01.01.2020']-1
df_financial['return_2021'] = df_financial['adjusted_01.01.2022']/df_financial['adjusted_01.01.2021']-1

In [13]:
# downloading exchange price
ex_2017 = yf.download('^GSPC', '2017-01-01',progress=False).Close.values[0]
ex_2018 = yf.download('^GSPC', '2018-01-01',progress=False).Close.values[0]
ex_2019 = yf.download('^GSPC', '2019-01-01',progress=False).Close.values[0]
ex_2020 = yf.download('^GSPC', '2020-01-01',progress=False).Close.values[0]
ex_2021 = yf.download('^GSPC', '2021-01-01',progress=False).Close.values[0]
ex_2022 = yf.download('^GSPC', '2022-01-01',progress=False).Close.values[0]

# calculating exchange return
ex_return_2017 = ex_2018/ex_2017-1
ex_return_2018 = ex_2019/ex_2018-1
ex_return_2019 = ex_2020/ex_2019-1
ex_return_2020 = ex_2021/ex_2020-1
ex_return_2021 = ex_2022/ex_2021-1

In [14]:
# calculating excess returns
df_financial['excess_2017'] = df_financial['return_2017']-ex_return_2017
df_financial['excess_2018'] = df_financial['return_2018']-ex_return_2018
df_financial['excess_2019'] = df_financial['return_2019']-ex_return_2019
df_financial['excess_2020'] = df_financial['return_2020']-ex_return_2020
df_financial['excess_2021'] = df_financial['return_2021']-ex_return_2021

In [17]:
# dropping columns that won't be used for modelling
columns = ['return_2017', 'return_2018', 'return_2019', 'return_2020', 'return_2021', 'adjusted_01.01.2017',
       'adjusted_01.01.2018', 'adjusted_01.01.2019', 'adjusted_01.01.2020',
       'adjusted_01.01.2021', 'adjusted_01.01.2022', 'Market_cap_2017', 'Market_cap_2018',
       'Market_cap_2019', 'Market_cap_2020', 'Market_cap_2021',
       'Market_cap_2022', 'Total_revenue_2016', 'Total_revenue_2017',
       'Total_revenue_2018', 'Total_revenue_2019', 'Total_revenue_2020',
       'Total_revenue_2021']
df_financial.drop(columns=columns, inplace=True)

In [20]:
# adding a constant
df_financial['constant'] = [1 for i in range(df_financial.shape[0])]
df_financial.head()

,Company,Ticker,scores_2016,scores_2017,scores_2018,scores_2019,scores_2020,total_2016,total_2017,total_2018,...,Avg_market_cap_2018,Avg_market_cap_2019,Avg_market_cap_2020,Avg_market_cap_2021,excess_2017,excess_2018,excess_2019,excess_2020,excess_2021,constant
0,3M,MMM,1745,1535,1806,1809,2096,7537,8099,8215,...,125568.5,106199.5,101136.5,101591.5,0.174235,-0.099849,-0.331689,-0.134937,-0.220422,1
1,Abbott,ABT,1335,1913,918,1144,1602,5719,7288,5201,...,113176.0,140322.0,170604.0,214482.5,0.352591,0.322953,-0.067979,0.117818,0.036356,1
2,Accenture,ACN,822,919,802,865,210,2293,2526,1825,...,98052.0,116434.5,142782.0,191781.5,0.157630,0.003235,0.223355,0.097815,0.337726,1
3,Adobe,ADBE,80,99,124,185,222,409,499,616,...,105480.5,136152.5,189338.5,261092.5,0.523257,0.343251,0.169007,0.353799,-0.135080,1
4,AEScorporation,AES,2,165,524,518,199,117,1041,2125,...,8364.5,11394.0,14421.0,15916.0,-0.212385,0.453911,0.132814,0.058926,-0.209512,1


In [28]:
# standarizing scores
mean_score = np.asarray([df_financial['scores_%d'%(2016+i)].values for i in range(5)]).ravel().mean()
std_score = np.asarray([df_financial['scores_%d'%(2016+i)].values for i in range(5)]).ravel().std()

for i in range(5):
    df_financial['scores_%d'%(2016+i)] = (df_financial['scores_%d'%(2016+i)] - mean_score)/(std_score)

In [31]:
print(f"Mean: {mean_score}, Std: {std_score}")

Mean: 566.642, Std: 433.3465793519086


In [34]:
columns = ['Company', 'constant']
       
df_pivot = df_financial[columns]

newdf = pd.DataFrame(np.repeat(df_pivot.values, 5, axis=0))

year = np.asarray([[2017, 2018, 2019, 2020, 2021] for i in range(df_financial.shape[0])]).ravel()
newdf['year'] = year
newdf.columns = columns+['year']
newdf.head()

,Company,constant,year
0,3M,1,2017
1,3M,1,2018
2,3M,1,2019
3,3M,1,2020
4,3M,1,2021


In [37]:
df_financial.columns

Index(['Company', 'Ticker', 'scores_2016', 'scores_2017', 'scores_2018',
       'scores_2019', 'scores_2020', 'total_2016', 'total_2017', 'total_2018',
       'total_2019', 'total_2020', 'Revenue_growth_2017',
       'Revenue_growth_2018', 'Revenue_growth_2019', 'Revenue_growth_2020',
       'Revenue_growth_2021', 'Avg_market_cap_2017', 'Avg_market_cap_2018',
       'Avg_market_cap_2019', 'Avg_market_cap_2020', 'Avg_market_cap_2021',
       'excess_2017', 'excess_2018', 'excess_2019', 'excess_2020',
       'excess_2021', 'constant'],
      dtype='object')

In [38]:
df_fin = df_financial
scores = []
totals = []
market_caps = []
growths = []
returns = []
exchanges = []

for company in df_pivot['Company'].values:
       for i in range(5):
              score = df_fin[df_fin['Company']==company]['scores_%d'%(2016+i)].values[0]
              scores.append(score)
              total = df_fin[df_fin['Company']==company]['total_%d'%(2016+i)].values[0]
              totals.append(total)
              market_cap = df_fin[df_fin['Company']==company]['Avg_market_cap_%d'%(2017+i)].values[0]
              market_caps.append(market_cap)
              growth = df_fin[df_fin['Company']==company]['Revenue_growth_%d'%(2017+i)].values[0]
              growths.append(growth)
              return_ = df_fin[df_fin['Company']==company]['excess_%d'%(2017+i)].values[0]
              returns.append(return_)
newdf['score'] = scores
newdf['total'] = totals
newdf['market_cap'] = market_caps
newdf['growth'] = growths
newdf['excess_return'] = returns

In [40]:
newdf.head()

,Company,constant,year,score,total,market_cap,growth,excess_return
0,3M,1,2017,2.719205,7537,123796.0,0.051413,0.174235
1,3M,1,2018,2.234604,8099,125568.5,0.035000,-0.099849
2,3M,1,2019,2.859970,8215,106199.5,-0.019197,-0.331689
3,3M,1,2020,2.866892,7996,101136.5,0.001494,-0.134937
4,3M,1,2021,3.529180,9147,101591.5,0.098527,-0.220422


In [41]:
newdf.to_csv('df_financial.csv')